In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import numpy as np
import cv2
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

from ssd import build_ssd

%matplotlib inline
from matplotlib import pyplot as plt
from data import BaseTransform, VOC_CLASSES as labels
import imageio

In [2]:
def detect(frame,net, transform):
# #     pre-process the frame
#     x = cv2.resize(image, (300, 300)).astype(np.float32)
#     x -= (104.0, 117.0, 123.0)
#     x = x.astype(np.float32)
#     x = x[:, :, ::-1].copy()
# #     plt.imshow(x)
#     x = torch.from_numpy(x).permute(2, 0, 1)
#     xx = Variable(x.unsqueeze(0))     # wrap tensor in Variable for recognizability by autograd
#     if torch.cuda.is_available():
#         xx = xx.cuda()
#     y = net(xx)
    
    height,width = frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2,0,1)
    x = Variable(x.unsqueeze(0))
    y = net(x) #extracting important info from neural network 
    
    plt.figure(figsize=(10,10))
    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
#     plt.imshow(rgb_image)  # plot the image for matplotlib
    currentAxis = plt.gca()
    plt.close('all')
    
    
    detections = y.data #[batch, numberOfClasses, number of occurence of the class,(score,x0,y0,x1,y1) ] 
#     score acts as a threshold to keep or discard the occurence
    scale = torch.Tensor([width, height, width, height])
    for i in range(detections.size(1)):
        j = 0
        while(detections[0, i, j, 0]>=0.60):
            score = detections[0,i,j,0]
            label_name = labels[i-1]
            display_txt = '%s: %.2f'%(label_name, score)
            pt = (detections[ 0, i, j, 1:] * scale).numpy()
            pt = (detections[0,i,j,1:]*scale).cpu().numpy()
            coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
            color = colors[i]
            currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
            currentAxis.text(pt[0], pt[1], display_txt, bbox={'facecolor':color, 'alpha':0.5})
        j+=1
    return frame


In [3]:
# create the SSD neural network
net = build_ssd('test', 300, 21)    # initialize SSD
net.load_weights('../weights/ssd300_mAP_77.43_v2.pth')

# creating the transform
transform = BaseTransform(net.size,(104/256.0, 117/256.0, 123/256.0)) 
# net size-target size of image to feed to neural network
# tuple represents the numbers used to obtain the right scale for color values


#doing object detection on a video
reader = imageio.get_reader('jogging.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output.mp4', fps = fps)
for i, frame in enumerate(reader):
#     print(frame.shape)
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame)
    print(i)
writer.close()

/home/shreyagu/Documents/openCV_practice/udemy/object_recognition/ssd_github/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/shreyagu/Documents/openCV_practice/udemy/object_recognition/ssd_github/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


Loading weights into state dict...
Finished!


0
1


KeyboardInterrupt: 